## FastCampus PJT - Team 3
##### 심성식 김민수 심승현 이희상 이용기

### 라이브러리 관련 및 환경 셋업

###### 필요한 모듈 설치

In [1]:
# !pip install numpy
# !pip install pandas
# !pip install matplotlib
# !pip install scipy
# !pip install seaborn

###### 필요한 모듈 모두 불러오기

In [2]:
import os
import time
import numpy as np
import pandas as pd
from matplotlib import rc
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import re
import glob
import seaborn as sns
import datetime

###### 폰트 확인

In [3]:
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False
font_path = fm.findfont('AppleGothic')
if not font_path:
    print('Warning: AppleGothic font not found')
else:
    print("AppleGothic font found at ", font_path)

font_prop = fm.FontProperties(fname=font_path, size=12)


AppleGothic font found at  /System/Library/Fonts/Supplemental/AppleGothic.ttf


### 경로 설정

###### 폴더 생성 및 경로 variable 생성

In [4]:
# Creating folders and variables for data paths

dataset = "./dataset/"
GROUPED = dataset + "Grouped/"
FINAL_GROUP = dataset + "Grouped_Final/"

if not os.path.exists("./dataset"):
    os.mkdir("./dataset")

if not os.path.exists(dataset + "Grouped"):
    os.mkdir(dataset + "Grouped")

if not os.path.exists(dataset + "Grouped_Final"):
    os.mkdir(dataset + "Grouped_Final")

###### 파일 경로 불러올 딕셔너리 생성

In [5]:
# Adding up all file locations into a dictionary.

file_locations = {
    "dataset" : dataset,
    'grouped': GROUPED,
    'group_final' : FINAL_GROUP #,
    # to be added further
}

all_file_locations = {}
for key, value in file_locations.items():
    all_file_locations[key] = glob.glob(value + "*.csv")

'''

type all_file_locations["key"] to call
keys are "dataset", "grouped", "group_final", etc.; to be added further.

'''

'\n\ntype all_file_locations["key"] to call\nkeys are "dataset", "grouped", "group_final", etc.; to be added further.\n\n'

#### 준비

###### Definition 설정

In [6]:
# Local definitions - 1

def clean_course_name(course_name):
    '''
    course name 중 첫 구절이 ( 으로 시작하고, 
    ~ 로 스플릿한 구절 중 0번째 len이 7인 경우; 예시: "(220123"
    ) 이후의 course name을 return합니다.
    
    아래와 같이 사용해 주시면 편합니다.
    df["course_name"] = df["course_name"].apply(clean_course_name)

    '''
    if course_name.startswith("("):
        # Classify the date range to cleanse.
        if "~" in course_name:
            temp = course_name.split("~")[0]
            # if the value before ~ is in year format, remove the whole bracket.
            if len(temp) == 7:
                return course_name.split(")")[1]
            else:
                return course_name
        else:
            return course_name
    else:
        return course_name

In [7]:
# Local definitions - 2

def course_packages(row):
    '''
    패키지명을 return하는 코드입니다.
    
    아래와 같이 apply 해주시고 적용 희망하시는 row를 대치시켜주시면 됩니다.
    df["course_group"] = df.apply(course_packages, axis=1)
    
    '''
    # Splitting course classifier (: and -) to extract package name
    parts = re.split("[:|-]", row["course_name"])
    # Finding course names without package classifier
    if len(parts) <= 1:
        # [무료강의] as an exception, it makes more sense if we classify itself alone.
        if parts[0].startswith("[무료강의]"):
            return "[무료강의]"
        # Classify all other courses without package classifier into Other.
        return ("Other")
    # Finding course names with package classifier.
    elif len(parts) > 1:
        # Classify [kit] first, as it needs to be separated from other [ ] covered words.
        if parts[0].startswith("[kit]"):
            return "[kit]"
        # Removing (( ))
        elif parts[0].startswith("(("):
            return parts[0].split("))")[1]
        # Removing ( )
        elif parts[0].startswith("("):
            return parts[0].split(")")[1]
        # Removing [ ]
        elif parts[0].startswith("["):
            return parts[0].split("]")[1]
        else:
            return parts[0]
    else:
        return parts[0]


In [ ]:
# Local definitions - 2

def coupons(df):
    '''
    coupon name을 통해 특수 형태의 쿠폰 분류,
    예외의 경우, 타 column 활용하여 쿠폰 분류
    
    아래와 같이 사용해 주시면 편합니다.
    df = coupons(df)
    
    '''
    # Find the length of DataFrame
    for i in range(len(df)):
        # Counter Set
        found = False
        # if dict key is in coupon name, insert dict value in coupon type
        if isinstance(df["coupon_name"][i], str):
            for key, value in external_coupons.items():
                if key in df["coupon_name"][i]:
                    df.at[i, "coupon_type"] = value
                    # counter
                    found = True
                    break
            else:
                for key, value in internal_coupons.items():
                    if key in df["coupon_name"][i]:
                        df.at[i, "coupon_type"] = value
                        found = True
                        break
        # if not found from the above,
        if not found:
            # first, set all remaining coupon's type as general
            if isinstance(df["coupon_name"][i], str):
                df.at[i, "coupon_type"] = "general"
            # then, find 100% discount coupon and assign it as "free"
            if df["actual_sold_price"][i] == 0.0 and df["transaction_amount"][i] == 0.0:
                df.at[i, "coupon_type"] = "free"
            # then, find % discount coupons and assign it as its percent
            if isinstance(df["coupon_discount"][i], float) and isinstance(df["coupon_name"][i], str) and "%" in df["coupon_name"][i]:
                df.at[i, "coupon_type"] = f"{df['coupon_name'][i].split('%')[0][-2:]} percent"
            else:
                continue
    return df


###### 컬럼명 영문으로 변경 - Encoding 이슈 방지

In [8]:
# Changing column names from Korean to English

df = pd.read_csv('./eda-proj-fc-purchase.csv')
df.rename(columns={"거래id":"transaction_id", "유형":"l_type", "고객id":"customer_id","코스ID":"courseID","사이트":"site", "포맷":"l_format",
                   "카테고리":"l_categories", "코스(상품) 이름":"course_name", "거래일자":"transaction_date_time","쿠폰이름":"coupon_name",
                   "판매가격":"sold_price","결제수단":"payment_method","실거래금액":"actual_sold_price","쿠폰할인액":"coupon_discount",
                   "거래금액":"transaction_amount","환불금액":"refund_amount"
                   },
          inplace=True)
df.to_csv(dataset + "rawdata_eng.csv", index=False)

###### 날짜/시간 데이터 변환 및 분리

In [9]:
# Converting date_time to date and time.

rawdata = pd.read_csv(dataset+"rawdata_eng.csv")

df = rawdata['transaction_date_time'].str.split(" ", n=5, expand=True)
rawdata['Date'] = df[0].str.cat([df[1], df[2]], sep="")
rawdata['Time'] = df[3].str.cat([df[4]], sep=" ")

# Converting time to 24H format

rawdata["Time"] = rawdata["Time"].str.replace("오후", "PM").str.replace("오전","AM")
rawdata['Time'] = rawdata['Time'].apply(lambda x: datetime.datetime.strptime(x, '%p %I:%M:%S').strftime('%H:%M:%S'))
df = rawdata
# rawdata.to_csv(dataset + "rawdata.csv", index=False)

###### 불필요한 데이터 처리

In [10]:
# Deleting Rows

# 크리에이티브
print("Number of rows before deletion:", len(df))
df = df[df["l_categories"] != "크리에이티브"]
print("Number of rows after deletion:", len(df))

# Format 컬럼

df = df.drop("l_format", axis=1)
df = df.drop("l_type", axis=1)

# rowdata_eng.to_csv(dataset + "columns_deleted.csv", index=False)

Number of rows before deletion: 159328
Number of rows after deletion: 159327


###### 숫자 데이터 컬럼 형 변환 (Obj -> Float)

In [11]:
# Convert dtype
df = df.replace("-",np.nan)
df["sold_price"] = df["sold_price"].astype(float)
df["actual_sold_price"] = df["actual_sold_price"].astype(float)
df["coupon_discount"] = df["coupon_discount"].astype(float)
df["transaction_amount"] = df["transaction_amount"].astype(float)
df["refund_amount"] = df["refund_amount"].astype(float)
type_converted = df
type_converted.to_csv(dataset + "space_PLEASE.csv", index=False)

###### 코스 이름 컬럼 cleansing

In [12]:
# Removing date range in course_name

type_converted["course_name"] = type_converted["course_name"].apply(clean_course_name)
df_yes_space = type_converted
df_yes_space
# df_yes_space.to_csv(dataset +"df_yes_space.csv", index = False)

,transaction_id,customer_id,courseID,site,l_categories,course_name,transaction_date_time,coupon_name,sold_price,payment_method,actual_sold_price,coupon_discount,transaction_amount,refund_amount,Date,Time
0,2417084,20053,209016,FASTCAMPUS,업무 생산성,올인원 패키지 : 김왼손의 파이썬 업무자동화 유치원,2022. 12. 31. 오후 10:12:33,NaN,189000.0,TRANSFER,159000.0,NaN,159000.0,NaN,2022.12.31.,22:12:33
1,2415408,58309,210664,FASTCAMPUS,부동산/금융,초격차 패키지 : 한 번에 끝내는 부동산 금융(PF) 실무,2022. 12. 30. 오후 6:12:57,NaN,501000.0,TRANSFER,501000.0,NaN,501000.0,NaN,2022.12.30.,18:12:57
2,2413897,18075,211782,FASTCAMPUS,영상/3D,편집하는여자의 영상편집 마스터클래스 - 제 6강 다양한 효과를 응용하여 애프터이펙트...,2022. 12. 31. 오후 11:12:15,NaN,549000.0,CARD,549000.0,NaN,549000.0,NaN,2022.12.31.,23:12:15
3,2413891,41898,213450,FASTCAMPUS,프로그래밍,초격차 패키지 : 35개 프로젝트로 배우는 Android 앱 개발 feat. Jet...,2022. 12. 31. 오후 11:12:03,[WELCOME] 프로그래밍 3만원할인,165000.0,CARD,135000.0,30000.0,135000.0,NaN,2022.12.31.,23:12:03
4,2413887,94328,210773,FASTCAMPUS,프로그래밍,"초격차 패키지 : 개발자 취업 합격 패스 With 코딩테스트, 기술면접",2022. 12. 31. 오후 11:12:10,NaN,239000.0,CARD,239000.0,NaN,239000.0,NaN,2022.12.31.,23:12:10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159323,1551424,39658,204491,FASTCAMPUS,부업/창업,올인원 패키지 : 제 2의 월급을 만들어주는 직장인 스몰비즈니스 가이드,2022. 1. 1. 오전 12:01:08,[NEW22] 온라인 강의 1만원 즉시 추가 할인 쿠폰,217000.0,CARD,207000.0,10000.0,207000.0,NaN,2022.1.1.,00:01:08
159324,1551410,95136,206811,FASTCAMPUS,영상/3D,[Kit] 닷밀의 미디어아트 그래픽 클래스 : 모션그래픽 뉴패러다임 Creative...,2022. 1. 1. 오전 12:01:12,NaN,385000.0,CARD,385000.0,NaN,385000.0,NaN,2022.1.1.,00:01:12
159325,1551361,12764,202927,FASTCAMPUS,데이터 사이언스,올인원 패키지 : 파이썬을 활용한 데이터 전처리 Level UP,2022. 1. 1. 오전 12:01:21,[WELCOME] 패캠은 처음이지? 3만원 할인쿠폰,166000.0,CARD,136000.0,30000.0,136000.0,NaN,2022.1.1.,00:01:21
159326,1545070,19606,208104,FASTCAMPUS,업무 생산성,올인원 패키지 : 100억을 움직이는 50가지 PPT 디자인 원칙,2022. 1. 2. 오전 11:01:24,NaN,289000.0,VBANK,289000.0,NaN,289000.0,NaN,2022.1.2.,11:01:24


###### 패키지 별 분류

In [13]:
# Grouping courses by Packages

# "The Red" had many variances, needed to cleanse the course_name once again solely for this
df_yes_space["course_name"] = df_yes_space["course_name"].apply(lambda x: x.replace(" ", "").lower())
df_no_space = df_yes_space

# Applying def course_packages.
df_no_space["course_group"] = df_no_space.apply(course_packages, axis=1)
df_no_space.to_csv(dataset +"Packages_Separated.csv", index = False)

# Generating CSV file for each Package.
grouped_df = df_no_space.groupby("course_group")

for group_name, group_df in grouped_df:
    if group_name == "Other":
#     groupname = group_name.replace("/", "_")
        filename = f"{group_name}.csv"
        group_df.to_csv(GROUPED + f"{filename}", index=False)
    else:
        pass

###### 패키지 별 분류된 CSV파일 생성

In [14]:
# Some packages only had one course each, needed to re-classify them as others
for file in all_file_locations["grouped"]:
    df = pd.read_csv(file, low_memory=False)
    # print(file, file.split("/")[-1].split(".")[0] ,len(set(df["course_name"])))

df_new = pd.DataFrame()
for group_name, group_df in grouped_df:
    # We don't need to check Other.csv as we want to add more data to it
    if group_name == "Other":
        continue
    else:
        # Check if any Package has only one course
        if len(set(group_df["course_name"])) == 1:
            # There is only one course in [무료강의], but as it seemed more legit if we separate it from Others.
            if group_name == "[무료강의]":
                # Adding up [무료강의].csv to final CSV folder
                group_df.to_csv(FINAL_GROUP + "[무료강의].csv", index=False)
                pass
            else:
                # Add all packages that only has one course to df_new
                df_new = pd.concat([df_new, group_df])

        else:
            # Adding all the packages with multiple courses to the final CSV folder.
            filetitle = group_name.replace("/", "_")
            group_df.to_csv(FINAL_GROUP + f"{filetitle}.csv", index=False)

df_other = pd.read_csv(GROUPED + "Other.csv")
df_all = pd.concat([df_new, df_other])
df_all.to_csv(FINAL_GROUP + "Others.csv", index=False)

###### 쿠폰 분류 추가

In [97]:
internal_coupons = {"내부" : "internal",
                    "검수" : "internal",
                    "검토" : "internal",
                    "협업" : "internal",
                    "업무" : "internal",
                    "참고" : "internal",
                    "직원" : "internal",
                    "강사" : "internal",
                    "QA" : "internal",
                    "테스트" : "internal",
                    "조교" : "internal",
                    "test" : "internal",
                    "마케팅" : "internal",
                    "신해동" : "internal"
                   }
external_coupons = {"WELCOME" : "welcome",
                    "welcome" : "welcome",
                    "Welcome" : "welcome",
                     "페이백" : "payback",
                     "1+1" : "bogo",
                     "산학" : "univ",
                     "기수강자" : "return",
                     "미리보기" : "preview",
                     "100%": "free",
                     "무료" : "free"
                    }

In [103]:
df = pd.read_csv(dataset + 'final_prep_file.csv') #추가할 데이터 부르기

df = coupons(df)
df.to_csv(dataset+"fin.csv")

In [15]:
df_cp = pd.read_excel('./coupon_list_for_drop.xlsx')
# 쿠폰에서 키워드를 뽑아내서 카테고리화 하는 작업

input_cpname ='WELCOME'   # input_cpname은 쿠폰이름에서 특정 단어를 키워드로 검색하기 위해서
output_cpname = 'welcome'  # 쿠폰 분류명을 지정한다.


# 1- 해당 용어만 검색, 2- 해당 용어 검색중 null데이터만 - 1,2중 상황에 맞는 것을 선택해서 사용

# df_cp_cpname=df_cp[df_cp['쿠폰이름'].str.contains(input_cpname)]  #-1
df_cp_cpname = df_cp[df_cp['classification'].str.contains(input_cpname).isna()]  #-2

#----------------------------------------------------------------위 확인 아래 진행
# df_cp_cpname    #확인용
df_cp_cpname.to_excel('./temp.xlsx')   #엑셀로 뽑아서 확인하기위해서

#----------------------------------------------------------------위 확인 아래 진행

# 분류코드 입력 

for i in df_cp_cpname.index:
    df_cp.loc[i,'classification']=output_cpname


# #아래처럼 다양한 형식으로 분류 코드가 잘 들어갔는지 확인
# df_cp[df_cp['분류']==output_cpname]

# df_cp_cpname=df_cp[df_cp['쿠폰이름'].str.contains(input_cpname)]
# df_cp_cpname

#----------------------------------------------------------------위 확인 아래 진행

#추출
df_cp.to_excel('coupon_list_for_drop.xlsx', index=False)

In [16]:
# 쿠폰분류 카테고리 추가

#새로운 컬럼 추가null 형식으로
df = pd.read_csv(dataset + 'final_prep_file.csv') #추가할 데이터 부르기

new_coulnum=np.array([np.nan]*len(df))
df['coupon_type']=new_coulnum
df['coupon_type']=df['coupon_type'].astype('str')

df.info() #체크 관련 코드
#----------------------------------------------------------------위 확인 아래 진행
# 원본에 데이터 넣는 과정

for i in df.index:
    for j in df_cp.index:
        if df.loc[i,'coupon_name'] == df_cp.loc[j,'coupon_name']:
            df.loc[i,'coupon_type']=df_cp.loc[j,'classification']
            break

df.head(10) #체크 관련 코드
df.to_csv(dataset + "final_final.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159327 entries, 0 to 159326
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   transaction_id         159327 non-null  int64  
 1   customer_id            159327 non-null  int64  
 2   courseID               159327 non-null  int64  
 3   site                   159327 non-null  object 
 4   l_categories           159327 non-null  object 
 5   course_name            159326 non-null  object 
 6   transaction_date_time  159327 non-null  object 
 7   coupon_name            76381 non-null   object 
 8   sold_price             159268 non-null  float64
 9   payment_method         159327 non-null  object 
 10  actual_sold_price      159327 non-null  float64
 11  coupon_discount        76381 non-null   float64
 12  transaction_amount     148009 non-null  float64
 13  refund_amount          11318 non-null   float64
 14  Date                   159327 non-nu